<a href="https://colab.research.google.com/github/CeciliaMarson/CMLS-Homework1/blob/master/provawithpandasfunction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import librosa
import os
import matplotlib.pyplot as plt
import sklearn.svm
import IPython.display as ipd
import scipy as sp
import pandas as pd
from sklearn import datasets
import math
import csv

In [0]:
!wget --no-check-certificate -r "http://cvml.unige.ch/databases/DEAM/features.zip" -O "Input.zip" 

will be placed in the single file you specified.

--2020-04-18 07:56:59--  http://cvml.unige.ch/databases/DEAM/features.zip
Resolving cvml.unige.ch (cvml.unige.ch)... 129.194.69.1
Connecting to cvml.unige.ch (cvml.unige.ch)|129.194.69.1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 601522797 (574M) [application/zip]
Saving to: ‘Input.zip’

Input.zip           100%[===================>] 573.66M  26.7MB/s    in 22s     

2020-04-18 07:57:22 (25.7 MB/s) - ‘Input.zip’ saved [601522797/601522797]

FINISHED --2020-04-18 07:57:22--
Total wall clock time: 23s
Downloaded: 1 files, 574M in 22s (25.7 MB/s)


In [0]:
!wget --no-check-certificate -r "http://cvml.unige.ch/databases/DEAM/DEAM_audio.zip" -O "Input2.zip" 

will be placed in the single file you specified.

--2020-04-18 07:57:23--  http://cvml.unige.ch/databases/DEAM/DEAM_audio.zip
Resolving cvml.unige.ch (cvml.unige.ch)... 129.194.69.1
Connecting to cvml.unige.ch (cvml.unige.ch)|129.194.69.1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1343203527 (1.2G) [application/zip]
Saving to: ‘Input2.zip’

Input2.zip          100%[===================>]   1.25G  26.8MB/s    in 49s     

2020-04-18 07:58:12 (26.2 MB/s) - ‘Input2.zip’ saved [1343203527/1343203527]

FINISHED --2020-04-18 07:58:12--
Total wall clock time: 49s
Downloaded: 1 files, 1.2G in 49s (26.2 MB/s)


In [0]:
!wget --no-check-certificate -r "http://cvml.unige.ch/databases/DEAM/DEAM_Annotations.zip" -O "Input1.zip" 

will be placed in the single file you specified.

--2020-04-18 07:58:13--  http://cvml.unige.ch/databases/DEAM/DEAM_Annotations.zip
Resolving cvml.unige.ch (cvml.unige.ch)... 129.194.69.1
Connecting to cvml.unige.ch (cvml.unige.ch)|129.194.69.1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4735283 (4.5M) [application/zip]
Saving to: ‘Input1.zip’

Input1.zip          100%[===================>]   4.52M  5.08MB/s    in 0.9s    

2020-04-18 07:58:15 (5.08 MB/s) - ‘Input1.zip’ saved [4735283/4735283]

FINISHED --2020-04-18 07:58:15--
Total wall clock time: 1.2s
Downloaded: 1 files, 4.5M in 0.9s (5.08 MB/s)


In [0]:
!unzip Input.zip
!unzip Input1.zip
!unzip Input2.zip


Streaming output truncated to the last 5000 lines.
  inflating: annotations/annotations per each rater/dynamic (per second annotations)/arousal/585.csv  
  inflating: annotations/annotations per each rater/dynamic (per second annotations)/arousal/584.csv  
  inflating: annotations/annotations per each rater/dynamic (per second annotations)/arousal/582.csv  
  inflating: annotations/annotations per each rater/dynamic (per second annotations)/arousal/581.csv  
  inflating: annotations/annotations per each rater/dynamic (per second annotations)/arousal/58.csv  
  inflating: annotations/annotations per each rater/dynamic (per second annotations)/arousal/579.csv  
  inflating: annotations/annotations per each rater/dynamic (per second annotations)/arousal/577.csv  
  inflating: annotations/annotations per each rater/dynamic (per second annotations)/arousal/574.csv  
  inflating: annotations/annotations per each rater/dynamic (per second annotations)/arousal/568.csv  
  inflating: annotation

In [0]:
#df = pd.read_csv("/content/features/10.csv",sep=";", header=0, usecols=("frameTime","F0final_sma_stddev"))

array_str=[]
n_features=261
n_frame=88

for i in np.arange(2,1800):
  array_str.append(str(i)+".csv")

train_root = '/content/features/'
class_train_files = [f for f in os.listdir(train_root) if f in array_str]
n_train_samples1 = len(class_train_files)


a=class_train_files
for i in np.arange(0,n_train_samples1):
  for j in range(0, n_train_samples1-i-1):
    b=a[j]
    c=a[j+1]
    if int(b[:-4]) > int(c[:-4]) :
      a[j], a[j+1] = a[j+1], a[j]

class_train_files=a


train_features1 = np.zeros([n_train_samples1,n_features])
for index, f in enumerate(class_train_files):
  feat = pd.read_csv("/content/features/{}".format(f),sep=";", header=0)
  train_features1[index,:] = np.mean(feat.to_numpy(), axis=None)


In [0]:
len(train_features1)


1543

In [0]:
n_train_samples1=1543
annotations1 = pd.read_csv("/content/annotations/annotations averaged per song/song_level/static_annotations_averaged_songs_1_2000.csv",sep=",", header=0, nrows=n_train_samples1)
type(annotations1)

In [0]:
annotationdef=annotations1.to_numpy()
a_valence=annotationdef[:,2]
a_arousal=annotationdef[:,4]

In [0]:
df=np.zeros((n_frame,n_features))
df=pd.read_csv("/content/features/1900.csv",sep=";", header=0)

test=np.mean(df)


In [0]:
len(test)

261

In [0]:
feat_max=[]
feat_min=[]

X_train_normalized=np.zeros([n_train_samples1,n_features])
X_test_normalized=np.zeros([n_features])

for i in range(n_features):
  feat_max = np.max(train_features1[:,i], axis=0)
  feat_min = np.min(train_features1[:,i], axis=0)
  X_train_normalized[:,i] = (train_features1[:,i] - feat_min) / (feat_max - feat_min)
  X_test_normalized[i]= (test[i] - feat_min) / (feat_max - feat_min)
  


In [0]:



from sklearn.linear_model import LinearRegression
lr = LinearRegression(normalize=True)
lr1= LinearRegression(normalize=True)

lr.fit(X_train_normalized, a_valence)
lr1.fit(X_train_normalized, a_arousal)


prediction_v=lr.predict(np.transpose(X_test_normalized.reshape(-1,1)))
prediction_a=lr1.predict(np.transpose(X_test_normalized.reshape(-1,1)))

mean_v=np.mean(prediction_v)
mean_a=np.mean(prediction_a)




In [0]:
print(prediction_a)
print(prediction_v)

[5.94506497e+12]
[6.9075195e+12]
